In [6]:
import pandas as pd
import numpy as np
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV, train_test_split
import sklearn as sk
from sklearn.metrics import accuracy_score, roc_auc_score, f1_score

ModuleNotFoundError: No module named 'xgboost'

In [ ]:
X_train = pd.read_csv('0_X_train.csv', index_col='Id')
X_valid = pd.read_csv('1_X_valid.csv', index_col='Id')
X_test  = pd.read_csv('2_X_test.csv', index_col='Id')

y_train = pd.read_csv('0_y_train.csv', index_col='Id')
y_valid = pd.read_csv('1_y_valid.csv', index_col='Id')
y_test  = pd.read_csv('2_y_test.csv', index_col='Id')

num_vars = ['age', 'time_spent', 'banner_views', 'banner_views_old', 'days_elapsed_old', 'X4']

In [ ]:
target_weight = (len(y_train)-sum(y_train)) / sum(y_train)

In [ ]:
%%time
xgb_clf = XGBClassifier(tree_method="hist", enable_categorical=True)
param_search = {'n_estimators': [150, 200, 250],
                'learning_rate': [0.16, 0.17, 0.18, 0.19],
                'max_depth': range(2, 5),
                'colsample_bytree': [1],
                'gamma': [i/10.0 for i in range(1, 4)],
                'scale_pos_weight':[.7, .8, 1]}
model = GridSearchCV(xgb_clf, scoring='accuracy', param_grid=param_search, return_train_score=True, verbose=1, cv=3)
model.fit(X_train, y_train)

In [ ]:
best_params=model.best_params_

In [ ]:
model.best_score_

In [ ]:
xgb_model = XGBClassifier(**best_params, tree_method='hist', enable_categorical=True)
xgb_model.fit(X_train, y_train)

In [ ]:
# Get a graph
graph = xgb.to_graphviz(xgb_model, num_trees=1)
# Or get a matplotlib axis
ax = xgb.plot_tree(xgb_model, num_trees=1)
# Get feature importances
xgb_model.feature_importances_

In [ ]:
dict(zip(X_train.columns, xgb_model.feature_importances_))

In [ ]:
train_preds = xgb_model.predict_proba(X_train)[:,1]
test_preds = xgb_model.predict_proba(X_test)[:,1]

In [ ]:
for c in [.1, .15, .2, .25, .30, .35, .40, .45, .5, .55, .6, .65, .7, .75, .8, .85, .9]:
  y_train_preds = (train_preds > c).astype(int)
  acc_train = accuracy_score(y_train, y_train_preds)
  y_test_preds = (test_preds > c).astype(int)
  acc_test = accuracy_score(y_test, y_test_preds)
  print(f"Model train/test accuracy for threshold {c}: {np.round(acc_train,3)}  {np.round(acc_test,3)}")

In [ ]:
for c in [ 0.37, .38, 0.39, .40, .41, .42, .43, .44, .45, .46, .47, .48, .49]:
  y_train_preds = (train_preds > c).astype(int)
  acc_train = accuracy_score(y_train, y_train_preds)
  y_test_preds = (test_preds > c).astype(int)
  acc_test = accuracy_score(y_test, y_test_preds)
  print(f"Model train/test accuracy for threshold {c}: {np.round(acc_train,4)}  {np.round(acc_test,4)}")

In [ ]:
thresh = 0.38

In [ ]:
best_params

In [ ]:
full_model = XGBClassifier(**best_params, tree_method="hist", enable_categorical=True)
X = df.drop(["Id", "subscription"], axis=1)
y = df.subscription
full_model.fit(X,y)

In [ ]:
preds = full_model.predict_proba(X)[:,1]
y_preds = (preds > thresh).astype(int)
acc = accuracy_score(y, y_preds)
print(f"Model train accuracy for threshold {thresh}: {np.round(acc,4)}")

In [ ]:
#score on test data
df_valid = pd.read_csv("/home/subscription/Subscriptions_test.csv")
for col in cat_cols:
  df_valid[col] = df_valid[col].astype("category")
X_valid = df_valid.drop(["Id"], axis=1)

In [ ]:
preds_valid = full_model.predict_proba(X_valid)[:,1]
y_valid_preds = (preds_valid > thresh).astype(int)

In [ ]:
df_submit = pd.DataFrame({"Id":df_valid.Id, "subscription":y_valid_preds})